# query 过滤

文本嵌入和向量相似性搜索通过理解文档的含义以及它们之间的相似性来帮助我们查找文档。然而，当需要基于指定范围对信息进行检索时（例如：查找在特定日期创建的所有文档，或者标记在特定类别下的文档），仅仅使用相似性查找的效果并不理想。

- **时间条件**："2019年上映的电影"
- **质量标准**："评分8分以上的电影"  
- **数值范围**："时长2小时以内的电影"
- **复合条件**："2020年后上映且评分7分以上的科幻电影"


这就是元数据过滤发挥作用的地方，因为它可以有效地处理这些结构化过滤器，允许用户根据特定属性（如时间、类别、评分等）筛选搜索结果。

需要注意的是，元数据通常作为文档的属性信息，本身信息含量有限，因此：
- 在构建向量嵌入时，我们主要对文档的核心内容进行建模
- 元数据（如时间、类别等）仅作为文档属性保存，不参与主要的向量检索过程


在前述第 3 节中我们介绍了如何在构建索引时引入元数据。在本节中，我们主要介绍在完成索引的构建后，如何进一步的通过 query 过滤来提升检索效果。

## 代码实现

### 1. 环境配置与依赖导入

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key\n"

In [ ]:
from langchain_community.embeddings import ZhipuAIEmbeddings

base_embeddings = ZhipuAIEmbeddings(
    model="embedding-3",
)


### 2. 构建带元数据的电影数据库

电影数据包含丰富的结构化信息（标题、年份、时长、评分等），是演示元数据过滤的理想场景。我们将构建一个综合性的电影向量数据库。

In [5]:
import uuid
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

docs = [
    Document(
        page_content="天地灵气孕育出一颗能量巨大的混元珠，元始天尊将混元珠提炼成灵珠和魔丸，灵珠投胎为人，助周伐纣时可堪大用；而魔丸则会诞出魔王，为祸人间。元始天尊启动了天劫咒语，3年后天雷将会降临，摧毁魔丸。太乙受命将灵珠托生于陈塘关李靖家的儿子哪吒身上。然而阴差阳错，灵珠和魔丸竟然被掉包。本应是灵珠英雄的哪吒却成了混世大魔王。调皮捣蛋顽劣不堪的哪吒却徒有一颗做英雄的心。然而面对众人对魔丸的误解和即将来临的天雷的降临，哪吒是否命中注定会立地成魔？他将何去何从？",
        metadata={"title": "哪吒之魔童降世", "year": 2019, "rating": 8.4, "duration": 110},
    ),
    Document(
        page_content="天劫之后，哪吒、敖丙的灵魂虽保住了，但肉身很快会魂飞魄散。太乙真人打算用七色宝莲给二人重塑肉身。但是在重塑肉身的过程中却遇到重重困难，哪吒、敖丙的命运将走向何方？",
        metadata={"title": "哪吒之魔童闹海", "year": 2025, "rating": 8.5, "duration": 144},
    ),
    Document(
        page_content="""姜子牙、姬发带队坚守西岐，家园保卫战一触即发！邓婵玉、闻仲奉商王殷寿之命，率魔家四将等殷商大军征伐西岐，西岐一方得殷郊、雷震子、杨戬、哪吒等相助，更聚全民之力守卫家园。兵戈相对、法术交锋，两大阵营掀起强强对决，关于“封神榜” 的争夺正在继续......""",
        metadata={"title": "封神第二部：战火西岐", "year": 2025, "rating": 6.1, "duration": 144},
    ),
    Document(
        page_content="""熊大、熊二、光头强意外地和来自未来世界的小亮一起穿越到100年后：世界发生巨大灾变，孢子植物全面侵占，人类在末日中艰难求生，整个地球危在旦夕！而这一切的罪魁祸首竟是......光头强？！
    100年前究竟发生了什么？小亮是敌是友？光头强为何背负恶名？人类命运最终会走向何方？熊强三人组能否穿透这层层迷雾，重启未来？""",
        metadata={"title": "熊出没·重启未来", "year": 2025, "rating": 7.1, "duration": 108},
    ),
]
ids = [str(uuid.uuid4()) for _ in docs]
vectorstore = Chroma.from_documents(documents=docs, ids=ids, embedding=base_embeddings)

### 3. 自动化Query过滤

为了精确控制检索范围，我们有两种过滤方式：
1. **手动过滤**：在检索时手动传入filter参数（参考 `3. 索引阶段/2.Metadata和权限体系`）
2. **智能过滤**：通过大模型自动生成过滤条件


手动过滤要求开发者了解元数据字段名和过滤语法，并能够精准识别用户query中的过滤意图，根据每一个意图对应的元数据构建对应的过滤条件。这种方式不仅开发复杂，还无法覆盖用户表达的多样性，比如"2小时以内"、"高分电影"等，也无法处理"最新电影"这类隐含条件。

大模型可以直接理解自然语言查询，自动识别过滤条件并转换为结构化查询。它支持多种表达方式，能理解隐含条件，并自动处理错误，为用户提供更智能友好的检索体验。

In [6]:
filter_dict = {
    "rating": 8.5,
}
results = vectorstore.similarity_search("电影", filter=filter_dict, k=3)

In [7]:
# 测试1：评分过滤
print("=" * 50)
print("测试 1: 评分过滤")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")

测试 1: 评分过滤

结果 1:
标题: 哪吒之魔童闹海
年份: 2025
评分: 8.5
时长: 144分钟



#### 3.1 智能过滤实现

为了充分利用大模型的理解能力，我们让大模型根据用户query自动生成相应的过滤条件。具体实现步骤：

1. 将元数据结构定义传递给大模型
2. 大模型分析用户query中的过滤需求
3. 自动生成对应的元数据过滤条件

我们使用 `SelfQueryRetriever` 来实现这一功能：

In [ ]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="title",
        description="电影标题",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="电影上映年份",
        type="integer",
    ),
    AttributeInfo(
        name="duration",
        description="电影时长",
        type="integer",
    ),
    AttributeInfo(
        name="rating", description="电影评分，范围1-10分", type="float"
    ),
]
document_content_description = "电影简介"
llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
)

### 4. 测试效果展示

我们来看看现在的检索效果如何。我们设计了一系列测试用例来验证智能查询过滤的效果：
- **评分过滤**：测试大模型能否正确识别"评分超过8.0分"这类数值比较条件

In [10]:
# 测试1：评分过滤
print("=" * 50)
print("测试 1: 评分过滤")
print("query: '我想看一部评分超过 8.0 分的电影'")
results = retriever.invoke("我想看一部评分超过 8.0 分的电影")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")

测试 1: 评分过滤
query: '我想看一部评分超过 8.0 分的电影'

结果 1:
标题: 哪吒之魔童闹海
年份: 2025
评分: 8.5
时长: 144分钟


- **时长过滤**：验证"2小时以内"等时间范围的识别能力


In [11]:
# 测试2：时长过滤
print("=" * 50)
print("测试 2: 时长过滤")
print("query: '我想看一部时长2小时以内的电影'")
results = retriever.invoke("我想看一部时长2小时以内的电影")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")

测试 2: 时长过滤
query: '我想看一部时长2小时以内的电影'

结果 1:
标题: 熊出没·重启未来
年份: 2025
评分: 7.1
时长: 108分钟


- **年份过滤**：检测特定年份的精确匹配功能

In [12]:
# 测试3：年份过滤
print("=" * 50)
print("测试 3: 年份过滤")
print("query: '我想看一部2025年上映的电影'")
results = retriever.invoke("我想看一部2025年上映的电影")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")

测试 3: 年份过滤
query: '我想看一部2025年上映的电影'

结果 1:
标题: 熊出没·重启未来
年份: 2025
评分: 7.1
时长: 108分钟


- **多重过滤**：同时包含年份、评分等多个条件的复杂查询


In [13]:
# 测试4：复合条件过滤
print("=" * 50)
print("测试4: 复合条件过滤")
print("查询: '推荐一部2025年评分7分以上的动画电影'")
results = retriever.invoke("推荐一部2025年评分7分以上的动画电影")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")

测试4: 复合条件过滤
查询: '推荐一部2025年评分7分以上的动画电影'

结果 1:
标题: 熊出没·重启未来
年份: 2025
评分: 7.1
时长: 108分钟


- **数值范围**：测试区间查询的处理能力

In [14]:
print("=" * 50)
print("测试 5: 数值范围过滤")
print("query: '我想看一部评分在6到8分之间的电影'")
results = retriever.invoke("我想看一部评分在6到8分之间的电影")
for i, doc in enumerate(results):
    print(f"\n结果 {i+1}:")
    print(f"标题: {doc.metadata['title']}")
    print(f"年份: {doc.metadata['year']}")
    print(f"评分: {doc.metadata['rating']}")
    print(f"时长: {doc.metadata['duration']}分钟")


测试 5: 数值范围过滤
query: '我想看一部评分在6到8分之间的电影'

结果 1:
标题: 熊出没·重启未来
年份: 2025
评分: 7.1
时长: 108分钟


可以看到，通过大模型自动生成过滤，可以精准进行相应的检索，准确满足了用户的需求。

参考文献：[How to do "self-querying" retrieval](https://python.langchain.com/docs/how_to/self_query/)